In [1]:
%store

Stored variables and their in-db values:
__importClassification             -> IPython.macro.Macro(u"# Handle table-like data and
__importRegression                 -> IPython.macro.Macro(u"# Handle table-like data and


In [1]:
%store -r __importRegression

In [2]:
__importRegression

/home/maviator/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
endMonth = 34
fnameTest = '../input/validation/test_' + str(endMonth) + '.csv'
fnameTrain = '../input/validation/train_' + str(endMonth) + '.csv'
#train = pd.read_csv(fnameTrain)
#test = pd.read_csv(fnameTest)
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
items = pd.read_csv("../input/items.csv")
categories = pd.read_csv("../input/item_categories.csv")

print (train.shape, test.shape, items.shape, categories.shape)

((2935849, 6), (214200, 3), (22170, 3), (84, 2))


labels = test.pop('item_cnt_day')

In [4]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [5]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [6]:
last_month_sales = train.loc[train.date_block_num == (endMonth-1)]
last_month_sales.reset_index(inplace=True)
print last_month_sales.shape
last_month_sales.head()

(53514, 7)


,index,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2882335,23.10.2015,33,45,13315,649.0,1.0
1,2882336,05.10.2015,33,45,13880,229.0,1.0
2,2882337,02.10.2015,33,45,13881,659.0,1.0
3,2882338,12.10.2015,33,45,13881,659.0,1.0
4,2882339,04.10.2015,33,45,13923,169.0,1.0


In [7]:
last_month_sales.pop('date')
last_month_sales.pop('date_block_num')
last_month_sales.pop('item_price')
last_month_sales.pop('index')
last_month_sales.head()

,shop_id,item_id,item_cnt_day
0,45,13315,1.0
1,45,13880,1.0
2,45,13881,1.0
3,45,13881,1.0
4,45,13923,1.0


In [8]:
last_month_sales.item_cnt_day.describe()

count    53514.000000
mean         1.327802
std          9.771091
min         -1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max       2169.000000
Name: item_cnt_day, dtype: float64

In [9]:
last_month_sales = last_month_sales.groupby(['shop_id', 'item_id'], as_index=False).agg({'item_cnt_day':sum})

In [10]:
print last_month_sales.shape
last_month_sales.head()

(31531, 3)


,shop_id,item_id,item_cnt_day
0,2,31,1.0
1,2,486,3.0
2,2,787,1.0
3,2,794,1.0
4,2,968,1.0


In [11]:
last_month_sales.drop_duplicates(inplace=True)

In [12]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


predictions = np.zeros(shape=test.shape[0])
print predictions.shape
print test.shape[0]
for i in range(test.shape[0]):
    #id0 = test['ID'][i]
    shop_id0 = test['shop_id'][i]
    item_id0 = test['item_id'][i]
    #print id0, shop_id0, item_id0
    if shop_id0 in last_month_sales.shop_id.values:
        possibles = last_month_sales.loc[last_month_sales.shop_id == shop_id0]
        if item_id0 in possibles.item_id.values: 
            t = possibles.loc[possibles.item_id==item_id0]
            predictions[i] = t.item_cnt_day.values[0]
        else:
            predictions[i] = 0.0
    else:
        predictions[i] = 0.0


In [13]:
test_id = test.pop('ID')

In [17]:
#train_labels = last_month_sales.pop('item_cnt_day')

lgbm = LGBMRegressor()
lgbm.fit(last_month_sales, train_labels)


train_predictions = lgbm.predict(last_month_sales)
predictions = lgbm.predict(test)

labels.shape

Clipping predictions in the [0, 20] range

In [18]:
for i in range(len(predictions)):
    if predictions[i] > 20:
        predictions[i] = 20
    if predictions[i] < 0:
        predictions[i] = 20

for i in range(len(labels)):
    if labels[i] > 20:
        labels[i] = 20
    if labels[i] < 0:
        labels[i] = 20

Compute RMSE

np.sqrt(mean_squared_error(labels, predictions))

In [19]:
np.sqrt(mean_squared_error(train_labels, train_predictions))

13.651431937035651

## Submission

In [22]:
test_submit = pd.DataFrame({'ID': test_id, 'item_cnt_month': predictions})
print test_submit.shape
test_submit.to_csv('lgbm_last_month.csv', index=False)
test_submit.head()

(214200, 2)


,ID,item_cnt_month
0,0,1.968267
1,1,1.968267
2,2,1.968267
3,3,1.968267
4,4,1.968267
